# [Java 程序员都需要懂的 反射！](https://mp.weixin.qq.com/s?__biz=MzI4Njg5MDA5NA==&mid=2247486658&idx=1&sn=58e682f3b9c68def4f12c574afac0de3&chksm=ebd74dc3dca0c4d542aa7ef2f7944dba438a447ffc2b39650dc4362698c6a7acd32e1a5fe3dc&token=2140209384&lang=zh_CN#rd)

## 序言
在学习Java基础的时候，一般都会学过反射。我在初学反射的时候，并不能理解反射是用来干嘛的。学了一些API发现：“明明我自己能直接new一个对象，为什么它要绕一个圈子，先拿到Class对象，再调用Class对象的方法来创建对象呢，这不是多余吗？”

## 引出Class对象
首先我们来看一段代码：

In [18]:
public class Demo {
    // 自建了一个Student类
    class Student{
    }
    public static void main() {
        // 将Object 强转成Student类
        Object o = new Object();
        Student s = (Student) o;
    }
}

In [17]:
Demo.main();

EvalException: class java.lang.Object cannot be cast to class REPL.$JShell$12$Demo$Student (java.lang.Object is in module java.base of loader 'bootstrap'; REPL.$JShell$12$Demo$Student is in unnamed module of loader jdk.jshell.execution.DefaultLoaderDelegate$RemoteClassLoader @646007f4)

编译成功，运行失败。
- 强转失败了

那么“Java”（实质上JVM）是怎么知道我们写的**强转**有没有问题的呢？可以依赖**Class**对象来协助判断。

- 一个`.java`的文件经过`javac`命令编译成功后，得到一个`.class`的文件
- 当我们执行了初始化操作(有可能是new、有可能是子类初始化 父类也一同被初始化、也有可能是反射…等)，会将`.class`文件通过**类加载器**装载到`jvm`中
- 将`.class`文件加载器加载到jvm中，又分了好几个步骤，其中包括 `加载、连接和初始化`
- 其中在加载的时候，会在Java堆中创建一个`java.lang.Class`类的对象，这个Class对象代表着**类相关的信息**。

既然说，Class对象代表着类相关的信息，那说明只要类有什么东西，在Class对象我都能找得到。

于是我们**可以通过Class对象来判断对象的真正类型**。

## 反射介绍
其实反射就是围绕着`Clas`s对象和`java.lang.reflect`类库来学习，就是各种的API

`Method/Field/Constructor`这些都是在`java.lang.reflect`类库下

**笔记**
```
想要使用反射，先要得到class文件对象，其实也就是得到Class类的对象
Class类主要API：
        成员变量  - Field
        成员方法  - Constructor
        构造方法  - Method
获取class文件对象的方式：
        1：Object类的getClass()方法
        2：数据类型的静态属性class
        3：Class类中的静态方法：public static Class ForName(String className)
--------------------------------  
获取成员变量并使用
        1: 获取Class对象
        2：通过Class对象获取Constructor对象
        3：Object obj = Constructor.newInstance()创建对象
        4：Field field = Class.getField("指定变量名")获取单个成员变量对象
        5：field.set(obj,"") 为obj对象的field字段赋值
如果需要访问私有或者默认修饰的成员变量
        1:Class.getDeclaredField()获取该成员变量对象
        2:setAccessible() 暴力访问  
---------------------------------          
通过反射调用成员方法
        1：获取Class对象
        2：通过Class对象获取Constructor对象
        3：Constructor.newInstance()创建对象
        4：通过Class对象获取Method对象  ------getMethod("方法名");
        5: Method对象调用invoke方法实现功能
如果调用的是私有方法那么需要暴力访问
        1: getDeclaredMethod()
        2: setAccessiable();          
```

## 为什么需要反射
主要有两个原因：
- 提高程序的灵活性
- 屏蔽掉实现的细节，让使用者更加方便好用

### 案例一（JDBC）

In [19]:
Class.forName("com.mysql.jdbc.Driver");

//获取与数据库连接的对象-Connetcion
connection = DriverManager.getConnection("jdbc:mysql://localhost:3306/java3y", "root", "root");

//获取执行sql语句的statement对象
statement = connection.createStatement();

//执行sql语句,拿到结果集
resultSet = statement.executeQuery("SELECT * FROM users");

EvalException: com.mysql.jdbc.Driver